In [1]:
import keras
from keras.models import model_from_json
model = model_from_json(open('model.json').read())
model.load_weights('weights.h5')
model.compile(loss='categorical_crossentropy', optimizer='adam')

Using Theano backend.
Using gpu device 0: GeForce GT 750M (CNMeM is enabled with initial size: 75.0% of memory, CuDNN 4007)


In [2]:
img_rows, img_cols = model.input_shape[1:]
print img_rows, img_cols

32 32


In [6]:
from IPython.display import clear_output
from time import time
from utils import show_array
import numpy as np
import zmq

bar_width = 50
display_rate = 1./10
addr = 'tcp://localhost:14000'

context = zmq.Context()
subs = context.socket(zmq.SUB)
subs.setsockopt(zmq.SUBSCRIBE, '')
subs.connect(addr)
start = time()
previous = 0
try:
    while True:
        raw = subs.recv()
        duration = (time() - start) % display_rate
        if duration < previous:
            data = np.fromstring(raw, dtype=np.uint8).reshape(img_rows, img_cols)
#             show_array(data)
            data = data.astype(np.float32).reshape(1, img_rows, img_cols) / 255.
            prediction = model.predict(data)[0]
            neutral_prob, smiling_prob = prediction
            weighted = smiling_prob / prediction.sum()
            left_side = ''.join(['-'] * int(bar_width * weighted))
            right_side = ''.join(['-'] * int(bar_width - (bar_width * weighted)))
            print 'Neutral', left_side, '###', right_side, 'Smiling'
            clear_output(wait=True)
        previous = duration
except:
    print 'Stopped'
    pass

Stopped
